Basic imports needed:

In [4]:
# from __future__ import print_function
# import collections
# tfe = tf.contrib.eager
# try:
#   tfe.enable_eager_execution()
# except ValueError:
#   pass

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

$\theta_0$ : Make a batch of bernoulli distributions and draw samples from it $G_0$.

In [51]:
theta_0 = tfd.Bernoulli(probs=[[0.9, 0.4],[0.3, 0.9]], name='theta_0')

G_0 = theta_0.sample()

G_0

<tf.Tensor 'theta_0/sample/Reshape:0' shape=(2, 2) dtype=int32>

In [6]:
sess = tf.Session()
G_0 = sess.run(G_0)
print(G_0)
# G_0

[[1 1]
 [1 1]]


Following method computes the Kronecker product two matrices. The convention is usually backwards that mat2 is scaled and block replicated on mat1 elementwise. Here lines are changed so that mat1 is scaled and block replicated on mat2. This is inline with the paper conventions.

In [34]:
def kronecker_product(mat1, mat2):
    m1 = tf.shape(mat1)[0]
    n1 = tf.size(mat1) // m1
    m2 = tf.shape(mat2)[0]
    n2 = tf.size(mat2) // m2
#     mat1_rsh =tf.reshape(mat1, [m1, 1, n1, 1])
    mat2_rsh =tf.reshape(mat1, [1, m1, 1, n1])
#     mat2_rsh =tf.reshape(mat2, [1, m2, 1, n2])
    mat1_rsh =tf.reshape(mat2, [m2, 1, n2, 1])
    return tf.reshape(mat1_rsh*mat2_rsh, [m1 * m2, n1 * n2])
#     return tf.reshape(tf.reshape(mat1_rsh * mat2_rsh, [m1 * m2, n1 * n2]),[-1])

An example run:

In [41]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = kronecker_product(a,b)

p = [[1.,0],[1.,0]]
q = [[3,4],[5.,6.]]
# c = sess.run(c, feed_dict={a:[[1.,0],[1.,0]], b:[[3.,4.],[5.,6.]]})
c = sess.run(c, feed_dict={a:p, b:q})
c

array([[ 3.,  0.,  4.,  0.],
       [ 3.,  0.,  4.,  0.],
       [ 5.,  0.,  6.,  0.],
       [ 5.,  0.,  6.,  0.]], dtype=float32)

Now make a batch of random Gaussian noise variables, given the matrix size: $\eta$

In [53]:
P_k = tf.placeholder(tf.float32)

mean = 0
std = 1
p_eta = tfd.Normal(loc=mean, scale=std)
eta = p_eta.sample(tf.shape(P_k))
probs = P_k + eta
# a = tf.constant(P_k)
# print(P_)

# Eta = tfd.Normal(loc=np.repmat([0], Theta), scale=, name='EdgeNoise_small')

print(sess.run(probs,feed_dict={P_k:[[1.,0],[1.,0]]}))

[[ 2.12771988  0.95698589]
 [ 0.21071374 -1.17736316]]
